In [0]:
dbutils.library.installPyPI('h2o-pysparkling-2.4')
from pysparkling import * # Import PySparkling
import h2o
import pandas as pd
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings("ignore")
from pysparkling.ml import H2OAutoML
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
from pyspark.sql.types import *
from datetime import datetime,date 

In [0]:
hc = H2OContext.getOrCreate()

Connecting to H2O server at http://10.139.64.7:9009 ... successful.
-------------------------- -------------------------------------------------------------------------------------------------------
H2O_cluster_uptime: 10 secs
H2O_cluster_timezone: Etc/UTC
H2O_data_parsing_timezone: UTC
H2O_cluster_version: 3.32.0.2
H2O_cluster_version_age: 16 days
H2O_cluster_name: sparkling-water-root_app-20201204023352-0000
H2O_cluster_total_nodes: 4
H2O_cluster_free_memory: 25.29 Gb
H2O_cluster_total_cores: 16
H2O_cluster_allowed_cores: 16
H2O_cluster_status: locked, healthy
H2O_connection_url: http://10.139.64.7:9009
H2O_connection_proxy: null
H2O_internal_security: False
H2O_API_Extensions: XGBoost, Algos, Amazon S3, Sparkling Water REST API Extensions, AutoML, Core V3, TargetEncoder, Core V4
Python_version: 3.7.3 final
-------------------------- -------------------------------------------------------------------------------------------------------

Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.2-1-2.4
 * H2O name: root
 * cluster size: 4
 * list of used nodes:
 (executorId, host, port)
 ------------------------
 (0,10.139.64.4,54321)
 (1,10.139.64.5,54321)
 (2,10.139.64.6,54321)
 (3,10.139.64.8,54321)
 ------------------------

 Open H2O Flow in browser: https://westeurope.azuredatabricks.net/driver-proxy/o/8767374559588946/1016-113926-refer971/9009/flow/index.html (CMD + click in Mac OSX)

In [0]:
def read_data():
  '''Read all locations' test data'''
  all_data_df=spark.table('churn.testing_data_2019_2020_all_loc')
  return all_data_df

def filter_data(all_data_df, location_id):
  '''Filter data by location id and prepare the dataframe for testing'''

  test_df=all_data_df[all_data_df.location_id==location_id]
  test_df_h20=test_df.drop('user_id','location_id','location_name','updated_on') # drop the columns not needed in train set
  test_df_h20=hc.asH2OFrame(test_df_h20)
  return test_df, test_df_h20

def load_model(location_name, model_update_date):
  location_name = location_name.replace(" ", "")
  try:
    path_tomodel="dbfs:/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_"+location_name+"_"+str(model_update_date[0])+".model/mojo_model/"
    print(path_tomodel)
    loaded_model=h2o.import_mojo(path_tomodel)
  except:
    path_tomodel="dbfs:/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_"+location_name+"_"+str(model_update_date[1])+".model/mojo_model/"
    print(path_tomodel)
    loaded_model=h2o.import_mojo(path_tomodel)
    
  #loaded_model=h2o.import_mojo('dbfs:/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_Bahrain_2020-11-18.model/mojo_model/')
  return loaded_model

def compute_levels(x):
    if(x>=median_value):
        return 'High Risk Churn'
    if(threshold_f1_max_metric<x<median_value):  #the value defined based on 25%quantiles and beigining of quantile values and also threshold metrics
      
        return 'Medium Risk Churn'
    if(x<=threshold_f1_max_metric):
        return 'Low Risk Churn'
      
def save_results(final_output):
  # updated_on=date.today()
  final_output['predictions_updated_on']=date.today() 
  #final_output = spark.createDataFrame(final_output)
  spark.createDataFrame(final_output).write.format('delta').mode('append').saveAsTable("churn.final_churn_testingdata_results_all_loc_2021")

In [0]:
def main_training_function(model_update_date):
  '''main function retrieving data for all locations, training the models and saving them'''
  
  # Get all the location ids and names
  inputs_df=spark.sql('select * from churn.all_location_inputs_train_test')
  all_locations_ids = [x["location_id"] for x in inputs_df.collect()]
  print('Total Locations=',len(all_locations_ids))
  print('ids: ',all_locations_ids)
  
  # Get all locations train data
  all_data_df=read_data()
    
  # make predictions seprately on test data for each location in this loop
  for loc_id in all_locations_ids:
  #if int(loc_id)>18:
    location_id = loc_id
    location_name = inputs_df.where(inputs_df.location_id == loc_id).select('location_name').collect()[0]['location_name']
    print(f'Start Predicting on Test Data For Location ID: {loc_id} and Location Name: {location_name}')

    # Filter and prepare test data by location_id
    test_df, test_df_h20=filter_data(all_data_df, location_id)

    # Load Trained Model 
    loaded_model=load_model(location_name, model_update_date)

    # Predictions
    predictions=loaded_model.predict(test_df_h20)
    test_predictions=predictions.as_data_frame()
    print('Predictions DF Shape: ',test_predictions.shape )
    print('Value Counts: ', test_predictions['predict'].value_counts())
    
    # Churn Risk Level
    median_value=test_predictions.p1.median()
    print('Median Value(p1): ', median_value)
    
    threshold_f1_max_metric=loaded_model.metric(metric='F1')[0][0]
    print('threshold_f1_max_metric: ', threshold_f1_max_metric)
    
    test_predictions['Level_of_ChurnRisk']=test_predictions['p1'].apply(lambda x: compute_levels(x))
    
    # Merge Predictions With Test Data
    test_df_pd = test_df.toPandas()
    final_output=pd.concat([test_df_pd,test_predictions],axis=1)
    print('Final Output Shape: ', final_output.shape)
    
    print('churn overview', final_output['Level_of_ChurnRisk'].value_counts(normalize=True)*100)
    
    # Save Predictions
    save_results(final_output)
    
  print('DONE....')

In [0]:
model_update_date=['2020-12-03', '2020-12-04']
main_training_function(model_update_date)

Total Locations= 13
ids: ['1', '2', '3', '4', '7', '8', '9', '10', '11', '14', '18', '20', '49']
Start Predicting on Test Data For Location ID: 1 and Location Name: Dubai
dbfs:/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_Dubai_2020-12-03.model/mojo_model/
generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
=============
H2OGenericEstimator : Import MOJO Model
Model Key: Generic_model_python_1607068669112_62

No model summary for this model

ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.1304220385244704
RMSE: 0.36113991544063695
LogLoss: 0.4147123727066082
Mean Per-Class Error: 0.17956098665858056
AUC: 0.9023709184648403
AUCPR: 0.8957801615299349
Gini: 0.8047418369296806

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4408392098641903: 
 0 1 Error Rate
----- ---- ---- ------- ---------------
0 4533 885 0.1633 (885.0/5418.0)
1 885 3618 0.1965 (885.0/4503.0)
Total 5418 4503 0.1784 (1770.0/9921.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric threshold value idx
--------------------------- ----------- -------- -----
max f1 0.440839 0.803464 213
max f2 0.231759 0.863883 309
max f0point5 0.575577 0.832924 157
max accuracy 0.483107 0.823506 196
max precision 0.961692 1 0
max recall 0.108148 1 385
max specificity 0.961692 1 0
max absolute_mcc 0.483107 0.643292 196
max min_per_class_accuracy 0.421306 0.815012 221
max mean_per_class_accuracy 0.448098 0.820439 209
max tns 0.961692 5418 0
max fns 0.961692 4445 0
max fps 0.0693138 5418 399
max tps 0.108148 4503 385
max tnr 0.961692 1 0
max fnr 0.961692 0.98712 0
max fpr 0.0693138 1 399
max tpr 0.108148 1 385

Gains/Lift Table: Avg response rate: 45.39 %, avg score: 46.70 %
group cumulative_data_fraction lower_threshold lift cumulative_lift response_rate score cumulative_response_rate cumulative_score capture_rate cumulative_capture_rate gain cumulative_gain kolmogorov_smirnov
------- -------------------------- ----------------- --------- ----------------- --------------- -------- -------------------------- ------------------ -------------- ------------------------- -------- ----------------- --------------------
1 0.0100796 0.960045 2.2032 2.2032 1 0.961183 1 0.961183 0.0222074 0.0222074 120.32 120.32 0.0222074
2 0.0200585 0.957683 2.2032 2.2032 1 0.958803 1 0.959999 0.0219853 0.0441928 120.32 120.32 0.0441928
3 0.0300373 0.955433 2.2032 2.2032 1 0.956642 1 0.958883 0.0219853 0.0661781 120.32 120.32 0.0661781
4 0.0400161 0.95301 2.2032 2.2032 1 0.954194 1 0.957714 0.0219853 0.0881634 120.32 120.32 0.0881634
5 0.0500958 0.949906 2.2032 2.2032 1 0.951777 1 0.956519 0.0222074 0.110371 120.32 120.32 0.110371
6 0.100091 0.926861 2.18987 2.19654 0.993952 0.940433 0.996979 0.948484 0.109483 0.219853 118.987 119.654 0.2193
7 0.150086 0.877965 2.10103 2.16473 0.953629 0.905373 0.982539 0.934123 0.105041 0.324895 110.103 116.473 0.320096
8 0.200081 0.805459 1.9811 2.11884 0.899194 0.842168 0.961713 0.911146 0.0990451 0.42394 98.1101 111.884 0.409912
9 0.300071 0.639273 1.76789 2.0019 0.802419 0.719949 0.908633 0.847435 0.176771 0.600711 76.7889 100.19 0.550508
10 0.40006 0.505043 1.43919 1.86126 0.653226 0.571087 0.844797 0.778365 0.143904 0.744615 43.9186 86.1255 0.63092
11 0.50005 0.389405 0.975004 1.68404 0.44254 0.444643 0.764362 0.711634 0.0974906 0.842105 -2.49961 68.4041 0.626343
12 0.60004 0.300757 0.672953 1.51555 0.305444 0.344194 0.687888 0.650405 0.0672885 0.909394 -32.7047 51.5554 0.566463
13 0.70003 0.23892 0.45974 1.36475 0.208669 0.26923 0.619438 0.595959 0.0459694 0.955363 -54.026 36.4745 0.467545
14 0.80002 0.186972 0.302051 1.23193 0.137097 0.211582 0.559153 0.547918 0.0302021 0.985565 -69.7949 23.1925 0.339755
15 0.90001 0.144404 0.119932 1.10838 0.0544355 0.165899 0.50308 0.505476 0.011992 0.997557 -88.0068 10.8384 0.17862
16 1 0.0676945 0.0244306 1 0.0110887 0.120365 0.453886 0.466969 0.00244282 1 -97.5569 0 0


ModelMetricsBinomialGeneric: generic
** Reported on 

In [0]:
predictions_df=spark.sql('select * from churn.final_churn_testingdata_results_all_loc_2021')
display(predictions_df)

user_id,total_owned_products,total_paid_for_product,num_distinct_merchant_ids,num_distinct_merchant_categories,num_days_before_endyear_active_redemption,diff_num_months_1st_last,avg_recency_days,median_monthly_spend,total_monthly_spend_redemptions,num_of_weeks_redemptions,num_of_months_redemptions,num_of_days_redemptions,count_of_total_redemptions,total_spend,gender,tenure_inapp,has_connect,has_redem_buffet,count_num_areas,min_spend_redem,max_spend_redem,median_spend_redem,median_num_days_permonth_active,avg_weekly_sessions_active,total_sessions_monthly,avg_monthly_sessions_active,avg_recency_days_inapp,num_days_before_endyear_active_session,num_distinct_months_active,num_distinct_days_active,total_sessions,advancesearch,categoryhome,favourites,home,merchantdetail,notifications,offers,offerslist,product,quicksearch,redemptioncard,num_days_before_end_purchased,offertime_of_purchase,tag_nationality,percentage_activity_of_redeem_appvisit,age_group,cor_tag,num_of_merchants_interacted,percentage_of_merchants_redeem_present,zero_redeemer,is_savings_morethan_paid,updated_on,location_id,location_name,predict,p0,p1,Level_of_ChurnRisk,predictions_updated_on
1576602,List(D20KSEPCP),220.52,22.0,1.0,3.0,10.0,5.0,209.0,285.0,29.0,11.0,36.0,62.0,2472.0,m,4.0,0.0,1.0,3.0,8.0,162.0,39.0,6.0,6.0,15.0,18.0,5.0,3.0,11.0,62.0,294.0,9.0,17.0,3.0,59.0,68.0,4.0,4.0,43.0,1.0,0.0,38.0,381.0,early_bird,other,58.0,35_45,local,125.0,22.0,0,1,09-11-2020,49,Eastern Province,1,0.2203751402714682,0.7796248597285318,High Risk Churn,2020-12-04
689172,List(D20KSEPCP),194.42,7.0,1.0,3.0,9.0,22.0,109.0,128.0,8.0,5.0,9.0,12.0,479.0,m,5.0,0.0,0.0,3.0,10.0,64.0,37.0,10.0,8.0,83.0,30.0,3.0,2.0,11.0,123.0,425.0,1.0,3.0,0.0,12.0,11.0,1.0,2.0,10.0,0.0,0.0,7.0,313.0,early_bird,IN,7.0,35_45,local,100.0,7.0,0,1,09-11-2020,49,Eastern Province,1,0.2641852099450545,0.7358147900549455,High Risk Churn,2020-12-04
4509275,List(D20KSEPCP),220.52,12.0,2.0,9.0,10.0,8.0,253.0,253.0,16.0,7.0,20.0,39.0,2042.0,m,2.0,0.0,1.0,2.0,8.0,195.0,37.0,5.0,2.0,55.0,9.5,5.0,9.0,10.0,62.0,190.0,11.0,29.0,1.0,84.0,35.0,2.0,26.0,35.0,1.0,2.0,23.0,380.0,early_bird,SA,32.0,45_55,local,51.0,12.0,0,1,09-11-2020,49,Eastern Province,1,0.12150613196415948,0.8784938680358405,High Risk Churn,2020-12-04
10881659,List(D20KSEPCP),123.45,5.0,1.0,46.0,2.0,7.0,50.0,437.0,6.0,3.0,8.0,10.0,514.0,m,0.0,0.0,1.0,3.0,8.0,175.0,23.5,2.0,2.0,1.0,7.0,10.0,47.0,3.0,7.0,16.0,0.0,0.0,12.0,75.0,81.0,12.0,0.0,25.0,0.0,0.0,50.0,114.0,other,SA,114.0,25_35,local,6.0,5.0,0,1,09-11-2020,49,Eastern Province,1,0.11609283121615865,0.8839071687838413,High Risk Churn,2020-12-04
364801,List(D20KSEPCP),338.14,14.0,1.0,2.0,10.0,13.0,143.0,190.0,11.0,8.0,16.0,23.0,3304.0,m,5.0,0.0,1.0,4.0,10.0,162.0,37.0,6.0,5.0,46.0,12.5,4.0,17.0,10.0,70.0,239.0,0.0,23.0,4.0,69.0,59.0,7.0,23.0,61.0,0.0,1.0,22.0,348.0,early_bird,SA,23.0,25_35,local,132.0,14.0,0,1,09-11-2020,49,Eastern Province,1,0.17270830996845155,0.8272916900315485,High Risk Churn,2020-12-04
5901430,List(D20KSEPCP),338.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,m,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0,2.0,14.0,253.0,3.0,4.0,7.0,43.0,43.0,0.0,100.0,29.0,0.0,43.0,29.0,0.0,0.0,0.0,346.0,early_bird,LB,0.0,55+,local,4.0,0.0,1,0,09-11-2020,49,Eastern Province,1,0.10185529642289526,0.8981447035771047,High Risk Churn,2020-12-04
8777359,List(D20KSEPCP),220.52,12.0,1.0,3.0,10.0,16.0,56.5,20.0,14.0,8.0,14.0,19.0,1081.0,m,1.0,0.0,1.0,3.0,10.0,215.0,21.0,2.0,4.0,12.0,9.5,12.0,3.0,8.0,25.0,78.0,1.0,24.0,3.0,77.0,46.0,12.0,19.0,40.0,4.0,3.0,28.0,382.0,early_bird,SY,56.0,25_35,local,28.0,12.0,0,1,09-11-2020,49,Eastern Province,1,0.4821396878305464,0.5178603121694536,Medium Risk Churn,2020-12-04
11132397,List(D20KSEPCP),123.53,6.0,1.0,4.0,1.0,5.0,166.0,166.0,4.0,3.0,7.0,9.0,700.0,m,0.0,0.0,1.0,4.0,13.0,166.0,37.0,7.0,8.0,19.0,19.0,2.0,4.0,3.0,23.0,64.0,31.0,0.0,3.0,61.0,61.0,2.0,0.0,16.0,0.0,0.0,42.0,42.0,other,SA,30.0,25_35,local,21.0,6.0,0,1,09-11-2020,49,Eastern Province,0,0.7490799364070593,0.25

In [0]:
# all_data_df=read_data()
# test_df, test_df_h20=filter_data(all_data_df, location_id=2)
# loaded_model=load_model(location_name='AbuDhabi', model_update_date='2020-12-03')
# predictions=loaded_model.predict(test_df_h20)
# test_predictions=predictions.as_data_frame()
# test_dff=all_data_df[all_data_df.location_id==3]
# df_test_pd=test_dff.toPandas()
# # df_test_pd.user_id.nunique()
# print(test_predictions['predict'].value_counts())
# median_value=test_predictions.p1.median()
# print(median_value)
# threshold_f1_max_metric=loaded_model.metric(metric='F1')[0][0]
# print(threshold_f1_max_metric)
# test_predictions['Level_of_ChurnRisk']=test_predictions['p1'].apply(lambda x: compute_levels(x))
# final_output=pd.concat([df_test_pd,test_predictions],axis=1)
# print(final_output['Level_of_ChurnRisk'].value_counts(normalize=True)*100)


dbfs:/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_AbuDhabi_2020-12-03.model/mojo_model/
generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
=============
H2OGenericEstimator : Import MOJO Model
Model Key: Generic_model_python_1607068669112_1

No model summary for this model

ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.1351023172674048
RMSE: 0.36756267121050906
LogLoss: 0.4291679552534939
Mean Per-Class Error: 0.18329872447646367
AUC: 0.9010055244903263
AUCPR: 0.9029584419233154
Gini: 0.8020110489806527

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.39659542374000495: 
 0 1 Error Rate
----- ---- ---- ------- ----------------
0 4064 1153 0.221 (1153.0/5217.0)
1 736 4098 0.1523 (736.0/4834.0)
Total 4800 5251 0.1879 (1889.0/10051.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric threshold value idx
--------------------------- ----------- -------- -----
max f1 0.396595 0.812692 235
max f2 0.271138 0.871731 299
max f0point5 0.524106 0.833333 179
max accuracy 0.465039 0.818128 204
max precision 0.943343 1 0
max recall 0.123067 1 382
max specificity 0.943343 1 0
max absolute_mcc 0.465039 0.636067 204
max min_per_class_accuracy 0.426325 0.815219 221
max mean_per_class_accuracy 0.465039 0.816701 204
max tns 0.943343 5217 0
max fns 0.943343 4830 0
max fps 0.0775788 5217 399
max tps 0.123067 4834 382
max tnr 0.943343 1 0
max fnr 0.943343 0.999173 0
max fpr 0.0775788 1 399
max tpr 0.123067 1 382

Gains/Lift Table: Avg response rate: 48.09 %, avg score: 47.64 %
group cumulative_data_fraction lower_threshold lift cumulative_lift response_rate score cumulative_response_rate cumulative_score capture_rate cumulative_capture_rate gain cumulative_gain kolmogorov_smirnov
------- -------------------------- ----------------- --------- ----------------- --------------- -------- -------------------------- ------------------ -------------- ------------------------- -------- ----------------- --------------------
1 0.0100488 0.935184 2.07923 2.07923 1 0.938277 1 0.938277 0.0208937 0.0208937 107.923 107.923 0.0208937
2 0.0200975 0.931636 2.07923 2.07923 1 0.933444 1 0.935861 0.0208937 0.0417873 107.923 107.923 0.0417873
3 0.0300468 0.9278 2.07923 2.07923 1 0.929819 1 0.93386 0.0206868 0.0624741 107.923 107.923 0.0624741
4 0.0400955 0.924061 2.05864 2.07407 0.990099 0.925905 0.997519 0.931866 0.0206868 0.0831609 105.864 107.407 0.0829693
5 0.0500448 0.920224 2.07923 2.0751 1 0.922073 0.998012 0.929919 0.0206868 0.103848 107.923 107.51 0.103656
6 0.10009 0.898409 2.07096 2.07303 0.996024 0.910186 0.997018 0.920053 0.103641 0.207489 107.096 107.303 0.206914
7 0.150035 0.855513 2.02124 2.05579 0.972112 0.87967 0.988727 0.90661 0.100952 0.30844 102.124 105.579 0.305182
8 0.20008 0.793685 1.89735 2.01616 0.912525 0.824872 0.969667 0.886165 0.0949524 0.403393 89.7349 101.616 0.3917
9 0.30007 0.645191 1.74614 1.92618 0.839801 0.719524 0.926393 0.830636 0.174597 0.577989 74.614 92.6184 0.535436
10 0.40006 0.517571 1.42753 1.80155 0.686567 0.577211 0.866451 0.767296 0.142739 0.720728 42.7531 80.1552 0.617795
11 0.50005 0.413801 1.06755 1.65478 0.513433 0.465387 0.795862 0.706926 0.106744 0.827472 6.75452 65.478 0.630807
12 0.60004 0.335026 0.76135 1.5059 0.366169 0.373009 0.724258 0.651282 0.0761274 0.9036 -23.865 50.5899 0.584834
13 0.70003 0.267032 0.469637 1.35788 0.225871 0.299776 0.65307 0.601074 0.046959 0.950559 -53.0363 35.7883 0.482665
14 0.80002 0.211559 0.29792 1.2254 0.143284 0.239249 0.589355 0.555852 0.029789 0.980348 -70.208 22.5404 0.347417
15 0.90001 0.159029 0.163442 1.10742 0.078607 0.185203 0.532611 0.514673 0.0163426 0.99669 -83.6558 10.7421 0.186263
16 1 0.0756336 0.0331022 1 0.0159204 0.131884 0.480947 0.476398 0.00330989 1 -96.6898 0 0


ModelMetricsBinomialGeneric: generic
** Reported on validation data. **

MSE: 0.17585592738388658
RMSE: 0.4193517943014988
LogLoss: 0.5268538612672156
Mean Per-Class Error: 0.280653717806109

In [0]:
# %fs ls 'dbfs:/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_Cape Town_2020-11-18.model/mojo_model/'


In [0]:
# final_output.head()

,user_id,total_owned_products,total_paid_for_product,num_distinct_merchant_ids,num_distinct_merchant_categories,num_days_before_endyear_active_redemption,diff_num_months_1st_last,avg_recency_days,median_monthly_spend,total_monthly_spend_redemptions,num_of_weeks_redemptions,num_of_months_redemptions,num_of_days_redemptions,count_of_total_redemptions,total_spend,gender,tenure_inapp,has_connect,has_redem_buffet,count_num_areas,min_spend_redem,max_spend_redem,median_spend_redem,median_num_days_permonth_active,avg_weekly_sessions_active,total_sessions_monthly,avg_monthly_sessions_active,avg_recency_days_inapp,num_days_before_endyear_active_session,num_distinct_months_active,num_distinct_days_active,total_sessions,advancesearch,categoryhome,favourites,home,merchantdetail,notifications,offers,offerslist,product,quicksearch,redemptioncard,num_days_before_end_purchased,offertime_of_purchase,tag_nationality,percentage_activity_of_redeem_appvisit,age_group,cor_tag,num_of_merchants_interacted,percentage_of_merchants_redeem_present,zero_redeemer,is_savings_morethan_paid,updated_on,location_id,location_name,predict,p0,p1,Level_of_ChurnRisk
0,559136,[D20BHBHCP],269.740000000000000000,7.0,2.0,6.0,10.0,25.0,78.0,156.0,10.0,7.0,11.0,12.0,582.0,m,5.0,1.0,0.0,5.0,20.0,78.0,44.0,6.0,6.0,25.0,12.0,5.0,6.0,11.0,59.0,152.0,5.0,22.0,15.0,69.0,57.0,14.0,9.0,41.0,3.0,0.0,26.0,381.0,early_bird,BH,19.0,25_35,local,203.0,7.0,0,1,09-11-2020,3,Bahrain,1,0.198406,0.801594,High Risk Churn
1,1334072,[D20BHBHCP],269.670000000000000000,4.0,1.0,82.0,6.0,36.0,49.0,8.0,4.0,4.0,4.0,5.0,224.0,m,4.0,0.0,0.0,3.0,8.0,78.0,59.0,3.0,4.0,10.0,10.0,12.0,82.0,7.0,19.0,75.0,15.0,29.0,1.0,76.0,27.0,5.0,16.0,24.0,11.0,8.0,7.0,259.0,early_bird,BH,21.0,25_35,local,15.0,4.0,0,0,09-11-2020,3,Bahrain,1,0.617522,0.382478,Low Risk Churn
2,1728288,[D20BHBHCP],269.670000000000000000,2.0,1.0,269.0,1.0,11.0,58.5,39.0,2.0,2.0,2.0,2.0,117.0,m,4.0,0.0,0.0,1.0,39.0,78.0,58.5,2.0,2.0,8.0,5.0,13.0,16.0,7.0,23.0,37.0,11.0,8.0,3.0,70.0,35.0,5.0,0.0,16.0,0.0,0.0,8.0,335.0,early_bird,BH,9.0,25_35,local,9.0,2.0,0,0,09-11-2020,3,Bahrain,1,0.090946,0.909054,High Risk Churn
3,8466077,[D20BHBHCP],269.720000000000000000,7.0,2.0,223.0,2.0,5.0,251.5,478.0,4.0,2.0,6.0,10.0,503.0,f,1.0,0.0,0.0,6.0,10.0,224.0,34.0,19.5,10.0,42.0,36.0,2.0,159.0,6.0,99.0,268.0,5.0,21.0,0.0,72.0,26.0,4.0,16.0,22.0,2.0,1.0,20.0,281.0,early_bird,PH,6.0,45_55,local,62.0,7.0,0,1,09-11-2020,3,Bahrain,1,0.234934,0.765066,Medium Risk Churn
4,5820597,[D20BHBHCP],269.670000000000000000,13.0,3.0,5.0,10.0,12.0,49.0,176.0,17.0,10.0,21.0,24.0,769.0,f,2.0,0.0,0.0,12.0,8.0,68.0,29.5,6.0,8.0,111.0,26.0,3.0,4.0,11.0,118.0,405.0,5.0,45.0,9.0,84.0,52.0,20.0,33.0,45.0,2.0,7.0,26.0,368.0,early_bird,BH,18.0,18_25,local,212.0,13.0,0,1,09-11-2020,3,Bahrain,0,0.691082,0.308918,Low Risk Churn


In [0]:
# %sql
# DROP TABLE IF EXISTS churn.final_churn_testingdata_results_all_loc_2021